# Spark: criando modelos de classificação

## 01. Preparando os dados
---

### O problema é a ferramenta

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Classificação com Spark") \
    .getOrCreate()

spark

25/09/03 14:22:30 WARN Utils: Your hostname, DSN-1003 resolves to a loopback address: 127.0.1.1; using 172.29.1.248 instead (on interface enp3s0)
25/09/03 14:22:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/03 14:22:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)

### Carregar e explorar os dados

In [3]:
dados = spark.read.csv(
    path="content/database/dados_clientes.csv",
    sep=",",
    header=True,
    inferSchema=True
)
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

25/09/03 14:22:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [5]:
dados.count()

10348

In [6]:
dados.groupBy("Churn").count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [7]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



### Tratando os dados

In [4]:
colunas_binarias = [
    "Churn",
    "Conjuge",
    "Dependentes",
    "TelefoneFixo",
    "MaisDeUmaLinhaTelefonica",
    "SegurancaOnline",
    "BackupOnline",
    "SeguroDispositivo",
    "SuporteTecnico",
    "TVaCabo",
    "StreamingFilmes",
    "ContaCorreio"
]

In [5]:
from pyspark.sql import functions as f

In [6]:
todas_colunas = [f.when(f.col(col) == "Sim", 1).otherwise(0).alias(col) for col in colunas_binarias]

In [7]:
[todas_colunas.insert(0, col) for col in reversed(dados.columns) if col not in colunas_binarias]
todas_colunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [8]:
dados.select(todas_colunas).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [9]:
dataset = dados.select(todas_colunas)

In [14]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



### Criando Dummies

In [15]:
dados.select(["Internet", "TipoContrato", "MetodoPagamento"]).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [16]:
dataset.groupBy("id").pivot("Internet").agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [10]:
internet = dataset.groupBy("id").pivot("Internet").agg(f.lit(1)).na.fill(0)
tipo_contrato = dataset.groupBy("id").pivot("TipoContrato").agg(f.lit(1)).na.fill(0)
metodo_pagamento = dataset.groupBy("id").pivot("MetodoPagamento").agg(f.lit(1)).na.fill(0)

In [11]:
dataset \
    .join(internet, "id", how="inner") \
    .join(tipo_contrato, "id", how="inner") \
    .join(metodo_pagamento, "id", how="inner") \
    .select(
        "*",
        f.col("DSL").alias("Internet_DSL"),
        f.col("FibraOptica").alias("Internet_FibraOptica"),
        f.col("Nao").alias("Internet_Nao"),
        f.col("Mensalmente").alias("TipoContrato_Mensalmente"),
        f.col("UmAno").alias("TipoContrato_UmAno"),
        f.col("DoisAnos").alias("TipoContrato_DoisAnos"),
        f.col("DebitoEmConta").alias("MetodoPagamento_DebitoEmConta"),
        f.col("CartaoCredito").alias("MetodoPagamento_CartaoCredito"),
        f.col("BoletoEletronico").alias("MetodoPagamento_BoletoEletronico"),
        f.col("Boleto").alias("MetodoPagamento_Boleto")
    ) \
    .drop(
        "Internet", "TipoContrato", "MetodoPagamento", "DSL",
        "FibraOptica", "Nao", "Mensalmente", "UmAno", "DoisAnos",
        "DebitoEmConta", "CartaoCredito", "BoletoEletronico", "Boleto"
    ) \
    .show()

25/09/03 14:23:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [12]:
dataset = dataset \
    .join(internet, "id", how="inner") \
    .join(tipo_contrato, "id", how="inner") \
    .join(metodo_pagamento, "id", how="inner") \
    .select(
        "*",
        f.col("DSL").alias("Internet_DSL"),
        f.col("FibraOptica").alias("Internet_FibraOptica"),
        f.col("Nao").alias("Internet_Nao"),
        f.col("Mensalmente").alias("TipoContrato_Mensalmente"),
        f.col("UmAno").alias("TipoContrato_UmAno"),
        f.col("DoisAnos").alias("TipoContrato_DoisAnos"),
        f.col("DebitoEmConta").alias("MetodoPagamento_DebitoEmConta"),
        f.col("CartaoCredito").alias("MetodoPagamento_CartaoCredito"),
        f.col("BoletoEletronico").alias("MetodoPagamento_BoletoEletronico"),
        f.col("Boleto").alias("MetodoPagamento_Boleto")
    ) \
    .drop(
        "Internet", "TipoContrato", "MetodoPagamento", "DSL",
        "FibraOptica", "Nao", "Mensalmente", "UmAno", "DoisAnos",
        "DebitoEmConta", "CartaoCredito", "BoletoEletronico", "Boleto"
    )

In [20]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [21]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

### Para saber mais: dados categóricos

Entender os tipos de dados que temos é uma tarefa muito importante, porque, através da identificação do dado, poderemos determinar quais são os melhores tratamentos de dados para serem utilizados em modelos de machine learning. Vamos analisar como diferenciar os dados categóricos binários e os não binários.

**Categóricos binários** são dados que só tem duas variações de valores, daí o nome binário, que é um sistema numérico de apenas dois números, o *um* e o *zero*. Mas os dados categóricos binários podem ser representados por outros valores que não o um e o zero. Por exemplo: em questionários, esses valores são registrados como *Sim* e *Não* ou *Verdadeiro* e *Falso*. Depois que conseguimos identificar que nosso dado é desse tipo, precisamos apenas transformar eles para os valores 0 e 1.

**Categóricos não binários**, também chamados de variáveis politômicas, são os que têm mais de dois valores possíveis. Nesses tipos de dados ainda temos mais uma ramificação categórica: **ordinais** e **nominais**.

- Os ordinais são quando os dados possuem uma ordem entre eles. Por exemplo, o grau de instrução de uma pessoa tem uma ordem: primeiro o ensino fundamental, depois o ensino médio e, finalmente o ensino superior.

- Os nominais são quando os valores não têm uma ordem, como: estado civil (solteiro, casado, separado, divorciado ou viúvo).

A identificação deste tipo é muito importante para determinarmos como vamos fazer a transformação desses dados para números. Para dados categóricos nominais, atribuir valores para as categorias não estaria correto, já que, dessa maneira, estaríamos aplicando uma ordem e um valor para as categorias que originalmente não tem.

No exemplo do estado civil, se atribuirmos os valores de 1, 2, 3, 4 e 5 para solteiro, casado, separado, divorciado e viúvo, respectivamente, podemos afetar o entendimento do modelo, pois podemos dizer que o viúvo é maior que solteiro, já que 5 é maior que 1, e essa não é uma informação correta. Por isso, quando identificamos esse tipo de dado, o procedimento mais usado será criar dummies para essa coluna, em que cada valor de categoria terá sua própria coluna de valores binários.

Percebeu como a análise e a identificação dos nossos dados é importante? Sendo assim, é bom evitar aplicar soluções automáticas para os tratamentos porque os resultados podem ser prejudicados.

### Para saber mais: Pandas ou Spark

Conhecemos duas ferramentas para trabalhar com dados em Python, o **Pandas** e o **Spark**. Mas como sabemos quando usar um ou o outro? Bom, no exemplo do nosso curso assumimos que o projeto terá uma grande base de dados e o Pandas, que carrega os dados em memória, provavelmente travaria ou funcionaria de forma bem lenta.

Para bases menores o Pandas poderia superar o Spark e entregar resultados bem mais rápidos. Mas, para entender melhor o porque disso acontecer, recomendo o Alura+ sobre o [Spark: RDD](https://cursos.alura.com.br/extra/alura-mais/spark-rdd-c1326), uma das representações de dados usada pelo Spark. Nesse conteúdo, você aprenderá sobre os RDDs e vai entender melhor sobre o funcionamento por trás do Pandas e do Spark.

Sabemos que questionamentos como, “*Pandas ou Spark: qual é o melhor?*”, nunca tem uma resposta simples, teremos sempre um “*depende*”. Então cabe a nós, cientistas de dados, conhecer ferramentas e soluções para conseguir entender quando elas terão o melhor resultado.

## 02. Criando o primeiro modelo
---

### Para saber mais: como funciona a Regressão Logística

O algoritmo de regressão logística faz parte dos algoritmos de aprendizado de máquina supervisionado e é utilizado para problemas nos quais a variável resposta é categórica, podendo ser do tipo **binomial** (apenas dois níveis) ou **multinomial** (três ou mais níveis).

Em nosso curso, vamos trabalhar com a variável categórica do tipo binomial. O intuito desse algoritmo é modelar a probabilidade de uma determinada classe. Para uma variável alvo com as classes 0 e 1, por exemplo, caso a probabilidade de um registro seja maior que um determinado ponto de corte, o modelo classifica esse registro como 1. Caso seja menor, o modelo classifica como 0.

Na imagem abaixo, podemos conferir como que a regressão logística se comporta:

![Regressão Logística](https://cdn3.gnarususercontent.com.br/2276-spark/02/Aula2-img1.jpg)

Na figura acima, os pontos em que y é igual a 1 representam os verdadeiros valores da classe 1. Os pontos em que y é igual a 0 representam os verdadeiros valores da classe 0. A curva em forma de S representa o modelo de regressão logística, que corresponde ao valor de probabilidade de cada um dos pontos.

O ponto de corte, definido como 0.5, faz com que os pontos representados em vermelho sejam classificados como da classe 0, uma vez que possuem valor de probabilidade segundo o modelo menor que 0.5. Já os pontos representados em azul são classificados como da classe 1, dado que a curva para esses pontos tem valor de probabilidade acima de 0.5.

### Preparação dos dados

In [22]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
dataset = dataset.withColumnRenamed("Churn", "label")

In [15]:
X = dataset.columns
X.remove("label")
X.remove("id")
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [16]:
assembler = VectorAssembler(inputCols=X, outputCol="features")

In [17]:
dataset_prep = assembler.transform(dataset).select("features", "label")

In [18]:
dataset_prep.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

### Ajuste e previsão

In [19]:
SEED = 101

In [20]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=SEED)

In [21]:
treino.count()

7206

In [22]:
teste.count()

3142

In [23]:
from pyspark.ml.classification import LogisticRegression

In [24]:
lr = LogisticRegression()

In [25]:
modelo_lr = lr.fit(treino)

25/09/03 14:24:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [26]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [37]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,

### Para saber mais: colunas rawPrediction e probability

Ao realizar os testes de predição do modelo com `modelo_lr.transform(treino)`, é retornada uma tabela. Ao utilizar o método `show()`, podemos ver sua estrutura:

<table>
    <thead>
        <tr>
            <th>features</th>
            <th>label</th>
            <th>rawPrediction</th>
            <th>probability</th>
            <th>prediction</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>(25, [0, 1, 2, 3, 4, 5,...</td>
            <td>0</td>
            <td>[2.90810234677711...</td>
            <td>[0.94824551456035...</td>
            <td>0.0</td>
        </tr>
        <tr>
            <td>(25, [0, 1, 2, 3, 4, 5,...</td>
            <td>0</td>
            <td>[2.51499120414495...</td>
            <td>[0.92518609960356...</td>
            <td>0.0</td>
        </tr>
        <tr>
            <td>(25, [0, 1, 2, 3, 4, 5,...</td>
            <td>0</td>
            <td>[0.51086016440655...</td>
            <td>[0.62500809542767...</td>
            <td>0.0</td>
        </tr>
    </tbody>
</table>

Além dos valores de características, os resultados esperados e o valor de predição, existem as colunas `rawPrediction` e `probability`. Essas duas colunas estão muito ligadas ao valor final de predição do modelo.

Os valores de `probability` são equivalentes aos valores da regressão depois de passados pela [função de ativação - função logística](https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_log%C3%ADstica) que abrange valores entre 0 e 1. O valor de predição (em `prediction`), nesse caso, é dado de acordo com o valor de `probability` de forma que:

- se `probability` maior ou igual a 0.5, `prediction` será `0`; e

- se `probability` menor que 0.5, `prediction` será `1`.

Os valores valores em `rawPrediction` são aqueles calculados antes de passarem pela função de ativação, ou seja, são os cálculos da regressão. Por conta dessas características, os valores das colunas `rawPrediction` e `probability` não têm utilidade para o objetivo do curso. Portanto, durante as análises, essas colunas serão ignoradas.

### Métricas

In [27]:
resumo_lr_treino = modelo_lr.summary

In [28]:
print(f"Acurácia: {resumo_lr_treino.accuracy}")
print(f"Precisão: {resumo_lr_treino.precisionByLabel[1]}")
print(f"Recall: {resumo_lr_treino.recallByLabel[1]}")
print(f"F1: {resumo_lr_treino.fMeasureByLabel()[1]}")

Acurácia: 0.7849014709963918
Precisão: 0.7706855791962175
Recall: 0.8125173082248685
F1: 0.7910488002156916


In [29]:
previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()

1256

In [30]:
tp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()
tn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 0)).count()
fp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 1)).count()
fn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 0)).count()
print(f"{tp=} {tn=} {fp=} {fn=}")

tp=1256 tn=1179 fp=400 fn=307


In [31]:
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize: bool = False, percentage: bool = True) -> None:
    tp = df_transform_modelo.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()
    tn = df_transform_modelo.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 0)).count()
    fp = df_transform_modelo.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 1)).count()
    fn = df_transform_modelo.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 0)).count()

    valorP = 1
    valorN = 1

    if normalize:
        valorP = tp + fn
        valorN = fp + tn

    if percentage and normalize:
        valorP = valorP / 100
        valorN = valorN / 100

    print(" " * 20, "Previsto")
    print(" " * 15, "Churn", " " * 5, "Não-Churn")
    print(" " * 4, "Churn", " " * 6, int(tp / valorP), " " * 7, int(fn / valorP))
    print("Real")
    print(" " * 4, "Não-Churn", " " * 2, int(fp / valorN), " " * 7, int(tn / valorN))

In [32]:
calcula_mostra_matriz_confusao(previsoes_lr_teste, normalize=True)

                     Previsto
                Churn       Não-Churn
     Churn        80         19
Real
     Não-Churn    25         74


## 03. Avaliando o novo modelo
---

### Para saber mais: como funciona a árvore de decisão?

Se você está se sentindo doente, mas não faz ideia do que possa ser e busca ajuda médica, é feita uma série de perguntas (dor de cabeça? febre? tosse?...). Ao final delas é emitido um parecer informando qual a causa dos sintomas. Nessa situação apresentada foi seguido o mesmo princípio utilizado em um estimador chamado **árvore de decisão**.

Árvore de decisão é um dos modelos de previsão mais simples, inspirado na forma que os seres humanos tomam as decisões e tem uma alta interpretabilidade, ou seja, uma compreensão fácil dos passos que foram realizados para conseguir chegar ao resultado final. Ela pode ser utilizada tanto para modelos de regressão, que têm intuito de prever valores numéricos, quanto para modelos de classificação, que têm intuito de prever categorias.

Graficamente, a árvore de decisão pode ser representada de forma que cada uma das decisões tomadas no processo possam ser visualizadas. Seus elementos principais são os **nós**, **ramos** e **folhas**. A estrutura da árvore se inicia com um **nó inicial**, também chamado de **raiz**. A partir dela são traçadas **ramificações**, que geram novos nós e o processo se repete para os nós subsequentes até que chegue a uma **folha**, que se trata de um nó especial que tem a informação da resposta, sendo ela uma categoria ou um valor previsto.

Cada ramo representa uma tomada de decisão a partir de um valor ou de uma categoria das variáveis explicativas, dividindo o conjunto de dados em nós que apresentam dados com características cada vez mais similares entre si.

Para que fiquem mais compreensíveis esses conceitos, vamos a um exemplo.

![Orientações sobre a COVID-19](https://cdn3.gnarususercontent.com.br/2276-spark/03/Aula3-img1.png)

Na imagem acima, temos uma orientação sobre Covid-19 para que as pessoas sejam direcionadas a ficarem isoladas ou não. A raiz ou nó inicial representa a pergunta se a pessoa está com sintomas de Covid-19, como aquelas perguntas realizadas pelo(a) médico(a) indicadas no início desse texto. A pergunta é respondida através dos ramos que partem da raiz, separando as pessoas que possuem sintomas das que não possuem.

O nó referente às pessoas que possuem sintomas se trata de um nó folha, com a decisão final de isolamento social. O nó referente às pessoas que não possuem sintomas se trata de um nó interno, que passa por um novo questionamento, criando assim novos ramos e nós. O processo se repete até que se chegue em decisões finais.

Portanto, o esquema se trata de uma árvore de decisão, em que é possível detectar todas as escolhas que foram feitas para se chegar à conclusões finais. O algoritmo de computador seguirá esses mesmos princípios, tomando as decisões com base nas variáveis explicativas.

#### Critério na divisão dos nós

Para conseguir identificar qual o melhor momento em que um nó deve ser dividido em dois ou mais subnós, o algoritmo da árvore de decisão considera alguns critérios. Os dois principais critérios de divisão usados nas árvores de decisão são:

#### Índice Gini

Este índice informa o **grau de heterogeneidade** dos dados. O objetivo dele é medir a frequência de um elemento aleatório de um nó ser rotulado de maneira incorreta. Em outros termos, esse índice é capaz de medir a impureza de um nó e ele é determinado por meio do seguinte cálculo:

$$\huge Gini = 1 - \displaystyle\sum_{i=1}^k P(i)^2$$

Onde:

- $\large P(i)$ representa a frequência relativa das classes em cada um dos nós;

- $\large k$ é o número de classes.

Se o índice Gini for igual a 0, isso indica que o nó é puro. No entanto, se o valor dele se aproxima mais do valor 1, o nó é impuro.

#### Entropia

A ideia básica da entropia é medir a **desordem dos dados** de um nó por meio da variável classificadora. Assim como o índice de Gini, ela é utilizada para caracterizar a impureza dos dados e pode ser calculada por meio da seguinte fórmula:

$$\huge Entropia(S) = \displaystyle\sum_{i=1}^c -p_{i} \log_{2} p_{i}$$

Onde:

- $\large p_{i}$ representa a proporção de dados no conjunto de dados $(S)$, pertencentes à classe específica $i$;
- $c$ é o número de classes.

### Ajuste e previsão

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

In [34]:
dtc = DecisionTreeClassifier(seed=SEED)

In [35]:
modelo_dtc = dtc.fit(treino)

In [36]:
previsoes_dtc_treino = modelo_dtc.transform(treino)
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [37]:
previsoes_dtc_treino.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,

In [38]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [62.0,128.0]|[0.32631578947368...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [51.0,141.0]| [0.265625,0.734375]|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [63.0,118.0]|[0.34806629834254...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,

### Métricas

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
evaluator = MulticlassClassificationEvaluator()

In [41]:
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"})

0.7917013599777962

In [42]:
print("Decision Tree Classifier")
print("=" * 40)
print("Dados de Treino")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_dtc_treino, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print()
print("=" * 40)
print("Dados de Teste")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_dtc_teste, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        2784         827
Real
     Não-Churn    674         2921
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.7917013599777962
Precisão: 0.8050896471949104
Recall: 0.7709775685405704
F1: 0.7876644504173151

Dados de Teste
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.7714831317632082
Precisão: 0.7785102175346078
Recall: 0.7555982085732565
F1: 0.7668831168831168


## 04. Comparação entre modelos
---

### Para saber mais: como funciona Random Forest

As árvores de decisão possuem uma característica que as impedem de serem consideradas como ferramenta ideal: a imprecisão. Isso quer dizer que elas funcionam muito bem com os dados utilizados para criá-las, mas não tão bem para realizar a classificação de novas amostras. A floresta aleatória (random forest) busca resolver esse problema de overfitting (sobreajuste).

O algoritmo Random Forest se baseia na utilização de diversas árvores de decisão para encontrar o resultado. Como essa árvore pode ser usada para regressão e classificação, o Random Forest também pode ser utilizado para os dois tipos de problemas. Vamos nos concentrar aqui nos problemas de classificação.

Para realizar a previsão, o algoritmo cria diversas árvores de decisão no conjunto de dados e é realizada a predição para cada uma delas. Internamente é feita uma “votação” para analisar qual predição tem maior ocorrência e, então, essa predição torna-se a resposta final. Se fosse utilizada a mesma base de dados na criação de todas as árvores de decisão do Random Forest, as respostas de cada uma das árvores seriam iguais e o resultado da votação seria idêntico a realizar um único modelo de árvore de decisão.

Para evitar esse problema, é utilizada uma técnica chamada de **bootstrapping**. Ela consiste em fazer amostragens com reposição do conjunto de dados original e cada uma delas será usada para uma árvore de decisão diferente. A amostragem com reposição significa que, ao sortearmos um elemento, isso não nos impede que ele mesmo apareça em sorteios futuros.

Dessa forma, as árvores terão resultados distintos, uma vez que são treinadas com conjuntos de dados diferentes. Na amostragem com repetição, as observações da tabela poderão ficar de fora e outras estarão duplicadas.

Para recapitular os passos:

- Ao utilizar o modelo Random Forest, podemos escolher a quantidade de árvores de decisão a serem criadas.

- O modelo irá criar um conjunto de dados para cada árvore a partir do método bootstrapping na ba de dados original, resultando em um resultado distinto para cada uma das árvores.

- Por fim, será feita uma votação entre os resultados das árvores e a classe predita na maioria das árvores é escolhida como a classificação do modelo Random Forest.

### Ajuste e Previsão

In [43]:
from pyspark.ml.classification import RandomForestClassifier

In [44]:
rfc = RandomForestClassifier(seed=SEED)

In [45]:
modelo_rfc = rfc.fit(treino)

In [46]:
previsoes_rfc_treino = modelo_rfc.transform(treino)

In [74]:
previsoes_rfc_treino.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[15.0052773466704...|[0.75026386733352...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[16.9295040273249...|[0.84647520136624...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[8.59288938528764...|[0.42964446926438...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.59647122885698...|[0.27982356144284...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.21616013157118...|[0.26080800657855...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[5.45640255581361...|[0.27282012779068...|       1.0|
|(24,[0,1,2,3,4,

### Métricas

In [47]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [76]:
previsoes_rfc_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[16.7433871675615...|[0.83716935837807...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.27313214599648...|[0.36365660729982...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[7.46885072161585...|[0.37344253608079...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.79829004739264...|[0.38991450236963...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.13263407834549...|[0.35663170391727...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[4.45872635511159...|[0.22293631775557...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.84691519125130...|[0.39234575956256...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.94796150783366...|[0.49739807539168...|       1.0|
|(24,[0,1,2,3,4,

In [48]:
print("Random Forest Classifier")
print("=" * 40)
print("Dados de Treino")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_rfc_treino, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print()
print("=" * 40)
print("Dados de Teste")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_rfc_teste, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        2950         661
Real
     Não-Churn    884         2711
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.7855953372189842
Precisão: 0.7694314032342201
Recall: 0.8169482137911935
F1: 0.7924781732706514

Dados de Teste
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.7702100572883513
Precisão: 0.7513448894202033
Recall: 0.8042226487523992
F1: 0.7768850432632881


## 05. Técnicas de otimização
---

### Para saber mais: o que são hiperparâmetros?

Cada um dos modelos de machine learning possui um comportamento distinto para prever os resultados utilizando métodos matemáticos e computacionais. O comportamento de um modelo dependerá de constantes, parâmetros ou características para que a fórmula matemática ou procedimento computacional se comporte de maneira diferente.

Esses argumentos que controlam o comportamento de um modelo de machine learning são chamados de **hiperparâmetros**. Ao alterar o valor desses parâmetros, modificamos também o desempenho do modelo, uma vez que, para cada conjunto de dados diferentes, é necessário um ajuste diferente dos hiperparâmetros para um modelo ser melhor adaptado a esse conjunto.

A árvore de decisão, por exemplo, possui uma característica chamada **profundidade** que diz respeito ao comprimento do caminho mais longo da raiz até uma folha da árvore. Uma profundidade muito grande permite que o modelo se ajuste melhor aos dados e classifique os registros de uma forma mais precisa.

Por outro lado, uma profundidade muito pequena pode fazer com que não existam muitas ramificações e os dados não sejam classificados de forma correta, pois o modelo não conseguiu “decidir” sobre como classificar os dados apresentados, por exemplo.

Desse modo, deve haver um equilíbrio no valor da profundidade, uma vez que se o modelo se ajustar perfeitamente aos dados de treinamento, não será capaz de generalizar para um conjunto de dados nunca visto, ocorrendo o que chamamos de overfitting (sobreajuste).

O modelo de árvore de decisão utitilizada para classificação, disponível na biblioteca [PySpark](https://spark.apache.org/docs/latest/ml-classification-regression.html), possui o parâmetro que controla a característica mencionada anteriormente chamado de `max_depth`, que é o hiperparâmetro que controla a profundidade da árvore.

Através da [documentação do DecisionTreeClassifier](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html), podemos verificar, dentro dos argumentos da função, os hiperparâmetros desse modelo:

```python
class pyspark.ml.classification.DecisionTreeClassifier(*, featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', rawPredictionCol='rawPrediction', maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0, maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity='gini', seed=None, weightCol=None, leafCol='', minWeightFractionPerNode=0.0)
```

Outro hiperparâmetro que pode ser controlado na árvore de decisão é o `minInstancesPerNode`, que está relacionado à quantidade mínima de amostras dos dados de treinamento para cada folha. Isso significa que, se há um nó na árvore com a quantidade de registros definida em `minInstancesPerNode`, este nó não pode ser dividido em outros nós. Ao utilizar um modelo sem definir um valor para os hiperparâmetros, os valores padrões serão utilizados. Em grande parte das vezes, não são as melhores opções para os dados.

### Árvore de decisão com CV

In [49]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [50]:
dtc = DecisionTreeClassifier(seed=SEED)

In [51]:
grid = ParamGridBuilder() \
    .addGrid(dtc.maxDepth, [2, 5, 10]) \
    .addGrid(dtc.maxBins, [10, 32, 45]) \
    .build()

In [52]:
evaluator = MulticlassClassificationEvaluator()

In [53]:
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED
)

In [54]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [56]:
previsoes_dtc_cv_treino = modelo_dtc_cv.transform(treino)
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [57]:
print("Decision Tree Classifier - Cross Validator")
print("=" * 40)
print("Dados de Treino")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_dtc_cv_treino, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_dtc_cv_treino, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_dtc_cv_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_dtc_cv_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_dtc_cv_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print()
print("=" * 40)
print("Dados de Teste")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier - Cross Validator
Dados de Treino
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        3255         356
Real
     Não-Churn    654         2941
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.8598390230363586
Precisão: 0.8326937835763623
Recall: 0.9014123511492661
F1: 0.865691489361702

Dados de Teste
Matriz de Confusão
                     Previsto
                Churn       Não-Churn
     Churn        1319         244
Real
     Não-Churn    430         1149
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.7854869509866327
Precisão: 0.7541452258433391
Recall: 0.8438899552143314
F1: 0.7964975845410629


### Random Forest com CV

In [58]:
rfc = RandomForestClassifier(seed=SEED)

In [59]:
grid = ParamGridBuilder() \
    .addGrid(rfc.maxDepth, [2, 5, 10]) \
    .addGrid(rfc.maxBins, [10, 32, 45]) \
    .addGrid(rfc.numTrees, [10, 20, 50]) \
    .build()

In [60]:
evaluator = MulticlassClassificationEvaluator()

In [61]:
rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED
)

In [62]:
modelo_rfc_cv = rfc_cv.fit(treino)

25/09/03 14:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1008.8 KiB
25/09/03 14:46:47 WARN DAGScheduler: Broadcasting large task binary with size 1375.8 KiB
25/09/03 14:46:57 WARN DAGScheduler: Broadcasting large task binary with size 1470.6 KiB
25/09/03 14:46:59 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/09/03 14:47:03 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/09/03 14:47:07 WARN DAGScheduler: Broadcasting large task binary with size 1834.6 KiB
25/09/03 14:47:27 WARN DAGScheduler: Broadcasting large task binary with size 1310.5 KiB
25/09/03 14:47:38 WARN DAGScheduler: Broadcasting large task binary with size 1462.2 KiB
25/09/03 14:47:40 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/09/03 14:47:44 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/09/03 14:47:48 WARN DAGScheduler: Broadcasting large task binary with size 1912.8 KiB
25/09/03 14:48:08 WARN DAGSchedul

In [63]:
previsoes_rfc_cv_treino = modelo_rfc_cv.transform(treino)
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [65]:
print("Random Forest Classifier - Cross Validator")
print("=" * 40)
print("Dados de Treino")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_rfc_cv_treino, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_rfc_cv_treino, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_rfc_cv_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_rfc_cv_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_rfc_cv_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print()
print("=" * 40)
print("Dados de Teste")
print("=" * 40)
print("Matriz de Confusão")
print("=" * 40)
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)
print("-" * 40)
print("Métricas")
print("-" * 40)
print("Acurácia:", evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão:", evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall:", evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1:", evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier - Cross Validator
Dados de Treino
Matriz de Confusão


25/09/03 14:57:45 WARN DAGScheduler: Broadcasting large task binary with size 1932.7 KiB
25/09/03 14:57:45 WARN DAGScheduler: Broadcasting large task binary with size 1932.7 KiB
25/09/03 14:57:46 WARN DAGScheduler: Broadcasting large task binary with size 1932.7 KiB
25/09/03 14:57:46 WARN DAGScheduler: Broadcasting large task binary with size 1932.7 KiB


                     Previsto
                Churn       Não-Churn
     Churn        3264         347
Real
     Não-Churn    628         2967
----------------------------------------
Métricas
----------------------------------------


25/09/03 14:57:47 WARN DAGScheduler: Broadcasting large task binary with size 1942.0 KiB


Acurácia: 0.8646960865945046


...

25/09/03 14:57:50 WARN DAGScheduler: Broadcasting large task binary with size 1942.0 KiB


Precisão: 0.7982035928143713


25/09/03 14:57:51 WARN DAGScheduler: Broadcasting large task binary with size 1942.0 KiB


Recall: 0.8528470889315419
F1: 0.8246210949582431


25/09/03 14:57:51 WARN DAGScheduler: Broadcasting large task binary with size 1942.0 KiB


In [66]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [68]:
print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees)

10
45
50


In [69]:
rfc_tuning = RandomForestClassifier(maxDepth=10, maxBins=45, numTrees=50, seed=SEED)

In [70]:
modelo_rfc_tuning = rfc_tuning.fit(dataset_prep)

25/09/03 15:02:40 WARN DAGScheduler: Broadcasting large task binary with size 1539.7 KiB
25/09/03 15:02:41 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/09/03 15:02:41 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


In [71]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [72]:
novo_cliente = [
    {
        "Mais65anos": 0,
        "MesesDeContrato": 1,
        "MesesCobrados": 45.30540797610398,
        "Conjuge": 0,
        "Dependentes": 0,
        "TelefoneFixo": 0,
        "MaisDeUmaLinhaTelefonica": 0,
        "SegurancaOnline": 0,
        "BackupOnline": 0,
        "SeguroDispositivo": 0,
        "SuporteTecnico": 0,
        "TVaCabo": 1,
        "StreamingFilmes": 1,
        "ContaCorreio": 1,
        "Internet_DSL": 1,
        "Internet_FibraOptica": 0,
        "Internet_Nao": 0,
        "TipoContrato_Mensalmente": 1,
        "TipoContrato_UmAno": 0,
        "TipoContrato_DoisAnos": 0,
        "MetodoPagamento_DebitoEmConta": 0,
        "MetodoPagamento_CartaoCredito": 0,
        "MetodoPagamento_BoletoEletronico": 1,
        "MetodoPagamento_Boleto": 0
    }
]

In [73]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [74]:
assembler = VectorAssembler(inputCols=X, outputCol="features")

In [75]:
novo_cliente_prep = assembler.transform(novo_cliente).select("features")

In [76]:
novo_cliente_prep.show()

+--------------------+
|            features|
+--------------------+
|(24,[1,2,11,12,13...|
+--------------------+



In [77]:
modelo_rfc_tuning.transform(novo_cliente_prep).show()

25/09/03 15:09:04 WARN DAGScheduler: Broadcasting large task binary with size 2027.9 KiB
25/09/03 15:09:04 WARN DAGScheduler: Broadcasting large task binary with size 2027.9 KiB
25/09/03 15:09:04 WARN DAGScheduler: Broadcasting large task binary with size 2027.9 KiB


+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[5.16473071594386...|[0.10329461431887...|       1.0|
+--------------------+--------------------+--------------------+----------+



In [78]:
spark.stop()